In [17]:
import tensorflow as tf
from datasets import load_dataset
from transformers import TFMT5ForConditionalGeneration, MT5Tokenizer, DataCollatorForSeq2Seq
from tensorflow.keras.optimizers import Adam


In [ ]:
tokenizer = MT5Tokenizer.from_pretrained("google/mt5-small")

In [19]:
from datasets import load_dataset, Dataset
import pandas as pd

ds = load_dataset("batubayk/TR-News")


Using the latest cached version of the dataset since batubayk/TR-News couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\Burak\.cache\huggingface\datasets\batubayk___tr-news\default\0.0.0\a125caa5d8ea8d27952f7fbe1b7f187ddf6a39bb (last modified on Wed Oct 30 21:48:28 2024).


In [20]:
# Only selecting the train dataset
train_ds = ds['train']

# Converting the object to DataFrame
dataset_new = pd.DataFrame(train_ds)

# https://tsmatz.wordpress.com/2022/11/25/huggingface-japanese-summarization/

# Her bir content sütunundaki kelime sayısını hesaplama
dataset_new['word_count'] = dataset_new['content'].apply(lambda x: len(x.split()))

# Ortalama kelime sayısını hesaplama
average_word_count = dataset_new['word_count'].mean()
average_word_count

240.87371970616738

In [21]:
train_ds

Dataset({
    features: ['abstract', 'author', 'content', 'date', 'source', 'tags', 'title', 'topic', 'url'],
    num_rows: 277573
})

In [22]:
len(dataset_new['content'][6].split()) 


913

In [23]:
def tokenize_sample_data(data):
    
    input_feature = tokenizer(data['content'], truncation=True, max_length=1024)
    label = tokenizer(data['abstract'], truncation=True, max_length=128)
    return {
        "input_ids" : input_feature['input_ids'],
        "attention_mask" : input_feature['attention_mask'],
        "labels" : label['input_ids'],
    }

tokenized_ds = ds.map(
    tokenize_sample_data,
    remove_columns= ['abstract','author','content','date','source','tags','title','topic','url'],
    batched=True,
    batch_size= 512  
)
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 277573
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 14610
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 15379
    })
})

In [24]:
'''

from transformers import MT5Tokenizer, MT5ForConditionalGeneration

# 1. Tokenizer ve modelin yüklenmesi
model_name = "google/mt5-small"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

# 2. Özetlenmek istenen Türkçe metin
turkish_text = """
İklim değişikliği, küresel sıcaklıkların artışıyla birlikte ortaya çıkan çevresel, sosyal ve ekonomik sorunlara neden olmaktadır.
Özellikle kuraklık, sel ve orman yangınları gibi doğal afetlerin sıklığı artarken, tarımsal üretimde de ciddi düşüşler yaşanmaktadır.
Bu durum, gıda güvenliğini tehdit etmekte ve toplumları olumsuz yönde etkilemektedir.
"""

# 3. Özetleme için giriş formatı: 'summarize:' ön ekini ekliyoruz
input_text = f"summarize: {turkish_text}"

# 4. Tokenizer ile metni encode etme
input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

# 5. Modeli kullanarak özetleme yapma
summary_ids = model.generate(
    input_ids, 
    max_length=150, 
    min_length=30, 
    length_penalty=2.0, 
    num_beams=4, 
    early_stopping=True
)

# 6. Özetlenen metni decode etme
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 7. Sonucu yazdırma
print("Özetlenen Metin:")
print(summary)
'''

'\n\nfrom transformers import MT5Tokenizer, MT5ForConditionalGeneration\n\n# 1. Tokenizer ve modelin yüklenmesi\nmodel_name = "google/mt5-small"\ntokenizer = MT5Tokenizer.from_pretrained(model_name)\nmodel = MT5ForConditionalGeneration.from_pretrained(model_name)\n\n# 2. Özetlenmek istenen Türkçe metin\nturkish_text = """\nİklim değişikliği, küresel sıcaklıkların artışıyla birlikte ortaya çıkan çevresel, sosyal ve ekonomik sorunlara neden olmaktadır.\nÖzellikle kuraklık, sel ve orman yangınları gibi doğal afetlerin sıklığı artarken, tarımsal üretimde de ciddi düşüşler yaşanmaktadır.\nBu durum, gıda güvenliğini tehdit etmekte ve toplumları olumsuz yönde etkilemektedir.\n"""\n\n# 3. Özetleme için giriş formatı: \'summarize:\' ön ekini ekliyoruz\ninput_text = f"summarize: {turkish_text}"\n\n# 4. Tokenizer ile metni encode etme\ninput_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)\n\n# 5. Modeli kullanarak özetleme yapma\nsummary_ids = model.genera

In [25]:
import torch
from transformers import AutoConfig, AutoModelForSeq2SeqLM
from transformers import TFAutoModelForSeq2SeqLM


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mt5_config = AutoConfig.from_pretrained(
    "google/mt5-small",
    max_length=128,
    length_penalty=0.6,
    no_repeat_ngram_size=2,
    num_beams=15,
)

# Load the model without the .to(device)
model = TFAutoModelForSeq2SeqLM.from_pretrained("google/mt5-small", config=mt5_config)


c:\Users\Burak\AppData\Local\Programs\Python\Python310\lib\site-packages\tf_keras\src\initializers\initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


In [26]:
data_collator = DataCollatorForSeq2Seq(
  tokenizer,
  model=model,
  return_tensors="pt")

In [27]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
  encoded_arg = tokenizer(arg)
  return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
  preds, labels = eval_arg
  # Replace -100
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  # Convert id tokens to text
  text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # Insert a line break (\n) in each sentence for ROUGE scoring
  # (Note : Please change this code, when you perform on other languages except for Japanese)
  text_preds = [(p if p.endswith(("!", "！", "?", "？", "。")) else p + "。") for p in text_preds]
  text_labels = [(l if l.endswith(("!", "！", "?", "？", "。")) else l + "。") for l in text_labels]
  sent_tokenizer_jp = RegexpTokenizer(u'[^!！?？。]*[!！?？。]')
  text_preds = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(p))) for p in text_preds]
  text_labels = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(l))) for l in text_labels]
  # compute ROUGE score with custom tokenization
  return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
  )

Using the latest cached version of the module from C:\Users\Burak\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--rouge\b01e0accf3bd6dd24839b769a5fda24e14995071570870922c71970b3a6ed886 (last modified on Wed Oct 30 23:15:32 2024) since it couldn't be found locally at evaluate-metric--rouge, or remotely on the Hugging Face Hub.


In [28]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="mt5-summarize-ja",
    log_level="error",
    num_train_epochs=5,  # Eğitim süresini kısaltmak gerekebilir
    learning_rate=3e-5,  # Daha düşük öğrenme oranı, büyük veri setlerinde genellikle daha iyidir
    lr_scheduler_type="linear",
    warmup_steps=1000,  # Daha fazla warmup adımı, büyük veri setleri için faydalı olabilir
    optim="adafactor",
    weight_decay=0.01,
    per_device_train_batch_size=8,  # Batch boyutunu artırmak daha iyi sonuç verebilir
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,  # Büyük bir batch boyutu elde etmek için gradient accumulation kullanımı
    evaluation_strategy="steps",
    eval_steps=500,  # Değerlendirme adımlarını artırma
    predict_with_generate=True,
    generation_max_length=128,
    save_steps=2000,  # Daha sık model kaydetme
    logging_steps=50,  # Daha az sık loglama
    push_to_hub=False,
)


c:\Users\Burak\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import TFSeq2SeqTrainer

trainer = TFSeq2SeqTrainer(
    model=model,  # Ensure this is a TensorFlow model
    args=training_args,
    data_collator=data_collator,
    compute_metrics=metrics_func,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["validation"].select(range(20)),
)

trainer.train()


C:\Users\Burak\AppData\Local\Temp\ipykernel_26972\190682653.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


AttributeError: 'TFMT5ForConditionalGeneration' object has no attribute 'to'